In [1]:
import sklearn
import pandas as pd


In [2]:
file = pd.read_excel(r'C:\Users\user\Desktop\time balance.xlsx')

In [3]:
df = pd.DataFrame(file)


In [4]:
print(df.isnull().sum())
df.dropna()


Major code stored  as  workover/drilling         0
Cde                                              0
Service hours type                               0
Minor Repair / Tripping / Complication      201202
dtype: int64


,Major code stored as workover/drilling,Cde,Service hours type,Minor Repair / Tripping / Complication
12,DTRP,6A,M/A y POOH TO SURF. B/OFF & L/DN DIR ASSY.,6A01
20,DTRP,6A,POOH.,6A02
23,DTRP,6A,M/A & R/I SAME BHA & TAGGED BOTTOM.,6A01
25,DTRP,6A,M/A & P/O 61 STDS + 02 SGLS OF 5” D/P.,6A02
28,DTRP,6A,M/A & P/O TO 1650M ( INSIDE CSG SHOE),6A01
...,...,...,...,...
291479,DTRP,6A,W/TRIP.,6A01
291483,PTRP,6A,WELL READY FOR PACKER SETTING JOB.,6A01
291486,PTRP,6A,POOH,6A02
291487,PTRP,6A,R/I WITH TCP GUN ASSY IS IN PROGRESS,6A01


In [5]:
import re
def preprocess_text(text):
   if isinstance(text,str):
      text = text.lower()  # Convert to lowercase
      text = re.sub(r'[^\w\s]', '', text)  # Remove punctuation
      text = re.sub(r'\s+', ' ', text).strip() #Remove extra-spaces
      return text




# Apply text preprocessing
df['Service hours type_lowercase'] = df['Service hours type'].apply(preprocess_text)

In [6]:
df['Service hours type_lowercase'] = df['Service hours type_lowercase'].apply(lambda x: str(x) if isinstance(x, str) else '')

In [7]:
import torch 
print(torch.cuda.is_available())

True


In [8]:
df.head()

,Major code stored as workover/drilling,Cde,Service hours type,Minor Repair / Tripping / Complication,Service hours type_lowercase
0,RDRP,1A,RIG CARRIER MOVED TO NEW SITE,NaN,rig carrier moved to new site
1,OUC,24J,DWS UNDER MAJOR REPAIR & WAITG FOR COMP OF CIV...,NaN,dws under major repair waitg for comp of civ job
2,OUC,24J,DWS UNDER MAJOR REPAIR & WAITG FOR COMP OF CIV...,NaN,dws under major repair waitg for comp of civ job
3,OUC,24J,DWS UNDER MAJOR REPAIR & WAITG FOR COMP OF CIV...,NaN,dws under major repair waitg for comp of civ job
4,OUC,24J,DWS UNDER MAJOR REPAIR & WAITG FOR COMP OF CIV...,NaN,dws under major repair waitg for comp of civ job


In [9]:
def discard_rare_samples(df, column_name, threshold=10):
    # Get the frequency of each unique value in the specified column
    value_counts = df[column_name].value_counts()

    # Filter the DataFrame to retain only the rows where the count is greater than or equal to the threshold
    fdf = df[df[column_name].isin(value_counts[value_counts >= threshold].index)]

    return fdf    

In [10]:
fdf = discard_rare_samples(df,'Cde',threshold =10)

In [11]:
fdf.reset_index(drop=True, inplace=True)

In [12]:
fdf.columns

Index(['Major code stored  as  workover/drilling', 'Cde', 'Service hours type',
       'Minor Repair / Tripping / Complication',
       'Service hours type_lowercase'],
      dtype='object')

In [13]:
fdf['Cde'].value_counts()

Cde
6A     67930
5A     38754
2A     22671
23A    13612
11A    13301
       ...  
23I       76
7B        70
11B       56
24K       45
21J       10
Name: count, Length: 73, dtype: int64

In [14]:
from sklearn.preprocessing import LabelEncoder

In [15]:
le = LabelEncoder()

In [17]:
fdf.loc[:, 'label_encoded'] = le.fit_transform(fdf['Cde'])

In [18]:
fdf.columns

Index(['Major code stored  as  workover/drilling', 'Cde', 'Service hours type',
       'Minor Repair / Tripping / Complication',
       'Service hours type_lowercase', 'label_encoded'],
      dtype='object')

In [19]:
from sklearn.model_selection import train_test_split


X = fdf.drop(columns=['label_encoded'])  # Features
y = fdf['label_encoded']  # Target

# Perform stratified split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, 
    test_size=0.3,  # 30% test set
    stratify=y,     # Ensure class distribution is preserved
    random_state=42
)

# Combine features and target back into DataFrames
train_df = X_train.copy()
train_df['label_encoded'] = y_train

test_df = X_test.copy()
test_df['label_encoded'] = y_test


In [20]:
# Verify class distribution
print(y_train.value_counts(normalize=True))
print(y_test.value_counts(normalize=True))


label_encoded
65    0.233045
64    0.132953
59    0.077778
41    0.046696
1     0.045633
        ...   
49    0.000260
67    0.000240
2     0.000191
58    0.000157
29    0.000034
Name: proportion, Length: 73, dtype: float64
label_encoded
65    0.233044
64    0.132949
59    0.077773
41    0.046703
1     0.045628
        ...   
49    0.000263
67    0.000240
2     0.000194
58    0.000149
29    0.000034
Name: proportion, Length: 73, dtype: float64


In [25]:
!pip install ipywidgets --upgrade
!jupyter nbextension enable --py widgetsnbextension

usage: jupyter [-h] [--version] [--config-dir] [--data-dir] [--runtime-dir]
               [--paths] [--json] [--debug]
               [subcommand]

Jupyter: Interactive Computing

positional arguments:
  subcommand     the subcommand to launch

options:
  -h, --help     show this help message and exit
  --version      show the versions of core jupyter packages and exit
  --config-dir   show Jupyter config dir
  --data-dir     show Jupyter data dir
  --runtime-dir  show Jupyter runtime dir
  --paths        show all Jupyter paths. Add --json for machine-readable
                 format.
  --json         output paths as machine-readable json
  --debug        output debug information about paths

Available subcommands: console contrib dejavu events execute kernel kernelspec
lab labextension labhub migrate nbconvert nbextensions_configurator notebook
qtconsole run script server troubleshoot trust

Jupyter command `jupyter-nbextension` not found.


In [24]:
!pip install jupyter-contrib-nbextensions


     ---------------------------------------- 0.0/23.5 MB ? eta -:--:--
     - -------------------------------------- 0.8/23.5 MB 5.6 MB/s eta 0:00:05
     ---- ----------------------------------- 2.6/23.5 MB 7.6 MB/s eta 0:00:03
     ------ --------------------------------- 3.9/23.5 MB 7.1 MB/s eta 0:00:03
     -------- ------------------------------- 4.7/23.5 MB 6.1 MB/s eta 0:00:04
     -------- ------------------------------- 5.2/23.5 MB 5.8 MB/s eta 0:00:04
     --------- ------------------------------ 5.8/23.5 MB 5.3 MB/s eta 0:00:04
     ---------- ----------------------------- 6.3/23.5 MB 4.7 MB/s eta 0:00:04
     ----------- ---------------------------- 6.6/23.5 MB 4.4 MB/s eta 0:00:04
     ------------ --------------------------- 7.1/23.5 MB 4.0 MB/s eta 0:00:05
     ------------ --------------------------- 7.3/23.5 MB 3.8 MB/s eta 0:00:05
     ------------- -------------------------- 7.9/23.5 MB 3.6 MB/s eta 0:00:05
     -------------- ------------------------- 8.4/23.5 MB 3

In [27]:
!jupyter contrib nbextension install --user


Traceback (most recent call last):
  File "D:\hehe\envs\pytorch\Lib\site-packages\jupyter_contrib_core\notebook_compat\nbextensions.py", line 6, in <module>
    from notebook.extensions import BaseExtensionApp
ModuleNotFoundError: No module named 'notebook.extensions'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "D:\hehe\envs\pytorch\Lib\site-packages\jupyter_contrib_core\notebook_compat\nbextensions.py", line 10, in <module>
    from notebook.nbextensions import BaseNBExtensionApp
ModuleNotFoundError: No module named 'notebook.nbextensions'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "D:\hehe\envs\pytorch\Scripts\jupyter-contrib.EXE\__main__.py", line 7, in <module>
  File "D:\hehe\envs\pytorch\Lib\site-packages\jupyter_core\application.py", line 28

In [33]:
from transformers import DistilBertTokenizer

tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')
tokens = tokenizer('Service hours type_lowercase')
print(tokens)


{'input_ids': [101, 2326, 2847, 2828, 1035, 2896, 18382, 102], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1]}


In [34]:
# Tokenize the training data
train_encodings = tokenizer(list(train_df['Service hours type_lowercase']), truncation=True, padding=True, max_length=128)

# Tokenize the testing data
test_encodings = tokenizer(list(test_df['Service hours type_lowercase']), truncation=True, padding=True, max_length=128)


In [35]:
import torch

class CustomDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

# Create PyTorch datasets
train_dataset = CustomDataset(train_encodings, list(train_df['label_encoded']))
test_dataset = CustomDataset(test_encodings, list(test_df['label_encoded']))

In [36]:
from transformers import DistilBertForSequenceClassification

# Number of labels in your classification task
num_labels = len(train_df['label_encoded'].unique())

# Load pre-trained DistilBERT model for sequence classification
model = DistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased', num_labels=num_labels)


Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [44]:
from sklearn.metrics import accuracy_score
import numpy as np

def compute_metrics(p):
    predictions = np.argmax(p.predictions, axis=1)
    return {
        'accuracy': accuracy_score(p.label_ids, predictions)
    }


In [45]:
from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir='./results',          # Directory to save model checkpoints and logs
    num_train_epochs=6,              # Number of training epochs
    per_device_train_batch_size=16,  # Batch size for training
    per_device_eval_batch_size=16,   # Batch size for evaluation
    warmup_steps=500,                # Number of warmup steps
    weight_decay=0.01,               # Weight decay
    logging_dir='./logs',            # Directory for logs
    logging_steps=10,                # Log every 10 steps
    evaluation_strategy='epoch',     # Evaluate after every epoch
    save_strategy='epoch',           # Save model checkpoints after every epoch
)


D:\hehe\envs\pytorch\Lib\site-packages\transformers\training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [46]:
from transformers import Trainer

trainer = Trainer(
    model=model,                         # The model to train
    args=training_args,                  # Training arguments
    train_dataset=train_dataset,         # Training dataset
    eval_dataset=test_dataset,           # Evaluation dataset
    compute_metrics=compute_metrics     # Compute metrics function
)


In [47]:
trainer.train()


D:\hehe\envs\pytorch\Lib\site-packages\torch\cuda\nccl.py:16: UserWarning: PyTorch is not compiled with NCCL support
  warnings.warn("PyTorch is not compiled with NCCL support")


Epoch,Training Loss,Validation Loss,Accuracy
1,0.812800,0.703586,0.830572
2,0.454800,0.686266,0.836678
3,0.303900,0.717147,0.838416
4,0.267500,0.788059,0.837227
5,0.144500,0.851082,0.838565
6,0.172100,0.896517,0.837627


D:\hehe\envs\pytorch\Lib\site-packages\torch\nn\parallel\_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
D:\hehe\envs\pytorch\Lib\site-packages\torch\cuda\nccl.py:16: UserWarning: PyTorch is not compiled with NCCL support
  warnings.warn("PyTorch is not compiled with NCCL support")
D:\hehe\envs\pytorch\Lib\site-packages\torch\nn\parallel\_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
D:\hehe\envs\pytorch\Lib\site-packages\torch\cuda\nccl.py:16: UserWarning: PyTorch is not compiled with NCCL support
  warnings.warn("PyTorch is not compiled with NCCL support")
D:\hehe\envs\pytorch\Lib\site-packages\torch\nn\parallel\_functions.py:68: UserWarning: Was asked 

TrainOutput(global_step=38262, training_loss=0.344430530271553, metrics={'train_runtime': 5381.3567, 'train_samples_per_second': 227.499, 'train_steps_per_second': 7.11, 'total_flos': 9197237808264456.0, 'train_loss': 0.344430530271553, 'epoch': 6.0})

In [50]:
# Save the tokenizer
tokenizer.save_pretrained(model_path)

('./saved_model\\tokenizer_config.json',
 './saved_model\\special_tokens_map.json',
 './saved_model\\vocab.txt',
 './saved_model\\added_tokens.json')

In [51]:
from transformers import DistilBertTokenizer

# Load the tokenizer from the saved directory
tokenizer = DistilBertTokenizer.from_pretrained(model_path)


In [52]:
from transformers import DistilBertForSequenceClassification, DistilBertTokenizer

# Define model path
model_path = './saved_model'

# Load the model and tokenizer
model = DistilBertForSequenceClassification.from_pretrained(model_path)
tokenizer = DistilBertTokenizer.from_pretrained(model_path)


In [53]:
from transformers import Trainer

# Initialize the Trainer
trainer = Trainer(
    model=model,                         # The model to train
    args=training_args,                  # Training arguments
    train_dataset=train_dataset,         # Training dataset
    eval_dataset=test_dataset,           # Evaluation dataset
    compute_metrics=compute_metrics      # Compute metrics function
)

# Evaluate the model on the test dataset
eval_results = trainer.evaluate()

# Print evaluation results
print("Evaluation results:", eval_results)


D:\hehe\envs\pytorch\Lib\site-packages\torch\nn\parallel\_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


Evaluation results: {'eval_loss': 0.8965165019035339, 'eval_accuracy': 0.8376273628597893, 'eval_runtime': 132.7013, 'eval_samples_per_second': 658.976, 'eval_steps_per_second': 20.595}


In [54]:
print(model.config)

DistilBertConfig {
  "_name_or_path": "./saved_model",
  "activation": "gelu",
  "architectures": [
    "DistilBertForSequenceClassification"
  ],
  "attention_dropout": 0.1,
  "dim": 768,
  "dropout": 0.1,
  "hidden_dim": 3072,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2",
    "3": "LABEL_3",
    "4": "LABEL_4",
    "5": "LABEL_5",
    "6": "LABEL_6",
    "7": "LABEL_7",
    "8": "LABEL_8",
    "9": "LABEL_9",
    "10": "LABEL_10",
    "11": "LABEL_11",
    "12": "LABEL_12",
    "13": "LABEL_13",
    "14": "LABEL_14",
    "15": "LABEL_15",
    "16": "LABEL_16",
    "17": "LABEL_17",
    "18": "LABEL_18",
    "19": "LABEL_19",
    "20": "LABEL_20",
    "21": "LABEL_21",
    "22": "LABEL_22",
    "23": "LABEL_23",
    "24": "LABEL_24",
    "25": "LABEL_25",
    "26": "LABEL_26",
    "27": "LABEL_27",
    "28": "LABEL_28",
    "29": "LABEL_29",
    "30": "LABEL_30",
    "31": "LABEL_31",
    "32": "LABEL_32",
    "33": "LABEL_33",
    "34": "LABEL_34",
    "